In [15]:
import json
from typing import List, Dict, Any, Union
from pydantic import BaseModel

class DynamicModelGenerator:
    def __init__(self, api_json: Dict[str, Any]):
        self.api_json = api_json
        self.models = {}

    def generate_pydantic_models(self):
        for item in self.api_json.get('item', []):
            group_name = item.get('name', 'UnnamedGroup')
            for subitem in item.get('item', []):
                endpoint_name = subitem.get('name', 'UnnamedEndpoint').replace(' ', '_')
                request = subitem.get('request', {})
                response = subitem.get('response', [])

                # Process request body
                if 'body' in request:
                    body_mode = request['body'].get('mode')
                    body_content = request['body'].get(body_mode, '{}')

                    if body_mode == 'raw':
                        try:
                            body_data = json.loads(body_content)
                            model_name = f"{group_name}_{endpoint_name}_Request"
                            self.models[model_name] = self.create_pydantic_model(model_name, body_data)
                        except json.JSONDecodeError:
                            pass

                # Process response body (if any examples are available)
                for resp in response:
                    try:
                        resp_body = json.loads(resp.get('body', '{}'))
                        model_name = f"{group_name}_{endpoint_name}_Response"
                        self.models[model_name] = self.create_pydantic_model(model_name, resp_body)
                    except json.JSONDecodeError:
                        pass

        return self.models

    @staticmethod
    def create_pydantic_model(name: str, data: Union[Dict[str, Any], List[Any]]) -> BaseModel:
        """Recursively creates a Pydantic model from a nested dictionary or list."""
        annotations = {}

        if isinstance(data, dict):
            for key, value in data.items():
                if isinstance(value, dict):
                    annotations[key] = (DynamicModelGenerator.create_pydantic_model(f"{name}_{key}", value), ...)
                elif isinstance(value, list) and value and isinstance(value[0], dict):
                    annotations[key] = (List[DynamicModelGenerator.create_pydantic_model(f"{name}_{key}_Item", value[0])], ...)
                else:
                    annotations[key] = (DynamicModelGenerator.infer_type(value), ...)

        elif isinstance(data, list) and data and isinstance(data[0], dict):
            return DynamicModelGenerator.create_pydantic_model(f"{name}_Item", data[0])

        # Return a dynamically created Pydantic model
        return type(name, (BaseModel,), {"__annotations__": annotations})

    @staticmethod
    def infer_type(value: Any) -> Any:
        """Infers the type of a value for Pydantic model fields."""
        if isinstance(value, str):
            return str
        elif isinstance(value, int):
            return int
        elif isinstance(value, float):
            return float
        elif isinstance(value, bool):
            return bool
        elif isinstance(value, list):
            return List[Any]
        elif value is None:
            return Union[None, Any]
        else:
            return Any

# Load JSON file
with open("/Users/clydeclarke/Documents/server-example-python-flask/app/tools/ClickUp API V2.postman_collection.json", 'r') as f:
    api_json = json.load(f)

# Generate models
generator = DynamicModelGenerator(api_json)
models = generator.generate_pydantic_models()

# Output generated models
for model_name, model in models.items():
    print(f"class {model_name}(BaseModel):")
    print(model.schema_json(indent=4))


PydanticSchemaGenerationError: Unable to generate pydantic-core schema for (<class 'str'>, Ellipsis). Set `arbitrary_types_allowed=True` in the model_config to ignore this error or implement `__get_pydantic_core_schema__` on your type to fully support it.

If you got this error by calling handler(<some type>) within `__get_pydantic_core_schema__` then you likely need to call `handler.generate_schema(<some type>)` since we do not call `__get_pydantic_core_schema__` on `<some type>` otherwise to avoid infinite recursion.

For further information visit https://errors.pydantic.dev/2.5/u/schema-for-unknown-type

In [7]:
api_json

{'info': {'_postman_id': '1c0ae0ce-6e1f-4f52-9708-296eead8ea2b',
  'name': 'ClickUp API V2',
  'schema': 'https://schema.getpostman.com/json/collection/v2.0.0/collection.json'},
 'item': [{'name': 'Attachments',
   'item': [{'name': 'Create Task Attachment',
     'request': {'method': 'POST',
      'header': [{'key': '',
        'name': 'Content-Type',
        'value': '',
        'type': 'text',
        'disabled': True},
       {'key': 'Content-Type', 'value': '', 'disabled': True}],
      'body': {'mode': 'formdata',
       'formdata': [{'key': 'attachment', 'type': 'file', 'src': []}]},
      'url': 'https://api.clickup.com/api/v2/task/task_id/attachment'},
     'response': []}]},
  {'name': 'Authorization',
   'item': [{'name': 'Get Access Token',
     'request': {'method': 'POST',
      'header': [],
      'url': {'raw': 'https://api.clickup.com/api/v2/oauth/token?client_id=&client_secret=&code=',
       'protocol': 'https',
       'host': ['api', 'clickup', 'com'],
       'path'